# Recurrent Neural Network Using Frameworks
---

In [1]:
import tensorflow as tf

## Hyperparameters

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

## Model Specification

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # 'Embedding': converts a very long one-hot vector (NUM_WORDS)
        # into a simple feature (in this case -> feature size: 16).
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        # SimpleRNN: vanilla RNN
        self.rnn = tf.keras.layers.SimpleRNN(32)
        self.dense = tf.keras.layers.Dense(2, activation = 'softmax')       
        
    def call(self, x, training = None, mask = None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

## Training and Test Loops

In [4]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training = True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(model, test_inputs, test_labels, loss_object, test_loss, test_accuracy):
    predictions = model(test_inputs, training = False)
    t_loss = loss_object(test_labels, predictions)
    test_loss(t_loss)
    test_accuracy(test_labels, predictions)

## Dataset

In [5]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = NUM_WORDS)

# setting inputs to the same length and adding padding if necessary
# using 'pad_sequence' in Keras
# for inputs -> pre-padding! (zero-padding)
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                       value = 0,
                                                       padding = 'pre',
                                                       # Use 'post' for the opposite cases.
                                                       maxlen = 32)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value = 0,
                                                       padding = 'pre',
                                                       maxlen = 32)
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Training Environment

In [6]:
# Creating a model
model = MyModel()

# Loss Function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# Optimization Algorithm
optimizer = tf.keras.optimizers.Adam()

# Performance Metrics
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')
test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

## Training

In [7]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer,
                  train_loss, train_accuracy)
        
    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object,
                 test_loss, test_accuracy)
        
    print(f'Epoch {epoch + 1}, ' +
         f'Loss: {train_loss.result()}, ' +
         f'Accuracy: {train_accuracy.result()}, ' +
         f'Test Loss: {test_loss.result()}, ' +
         f'Test_Accuracy: {test_accuracy.result()}')
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.5456036925315857, Accuracy: 0.7020400166511536, Test Loss: 0.45511192083358765, Test_Accuracy: 0.7832000255584717
Epoch 2, Loss: 0.3624837100505829, Accuracy: 0.8430799841880798, Test Loss: 0.48506003618240356, Test_Accuracy: 0.7707200050354004
Epoch 3, Loss: 0.23787671327590942, Accuracy: 0.9063599705696106, Test Loss: 0.6028680801391602, Test_Accuracy: 0.7651600241661072
Epoch 4, Loss: 0.11949918419122696, Accuracy: 0.9579600095748901, Test Loss: 0.7387099862098694, Test_Accuracy: 0.7469199895858765
Epoch 5, Loss: 0.05055755004286766, Accuracy: 0.9841600060462952, Test Loss: 0.9949736595153809, Test_Accuracy: 0.7430400252342224
Epoch 6, Loss: 0.02663850225508213, Accuracy: 0.9912800192832947, Test Loss: 1.195212960243225, Test_Accuracy: 0.7454400062561035
Epoch 7, Loss: 0.013936749659478664, Accuracy: 0.9962400197982788, Test Loss: 1.3949029445648193, Test_Accuracy: 0.75
Epoch 8, Loss: 0.018623070791363716, Accuracy: 0.9935200214385986, Test Loss: 1.4062968492507935,

The reason why the test results are not good enough seems to be due to the low feature dimension (in this case 16) and the low number of RNN layers. So simple RNN.

# LSTM (Long Short Term Memory)
---

## Model Specification

In [8]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        # just replacing SimpleRNN with LSTM
        self.rnn = tf.keras.layers.LSTM(32)
        self.dense = tf.keras.layers.Dense(2, activation = 'softmax')       
        
    def call(self, x, training = None, mask = None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

In [9]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer,
                  train_loss, train_accuracy)
        
    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object,
                 test_loss, test_accuracy)
        
    print(f'Epoch {epoch + 1}, ' +
         f'Loss: {train_loss.result()}, ' +
         f'Accuracy: {train_accuracy.result()}, ' +
         f'Test Loss: {test_loss.result()}, ' +
         f'Test_Accuracy: {test_accuracy.result()}')
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.0126925278455019, Accuracy: 0.995639979839325, Test Loss: 1.6280314922332764, Test_Accuracy: 0.7476800084114075
Epoch 2, Loss: 0.011911439709365368, Accuracy: 0.995959997177124, Test Loss: 1.6492708921432495, Test_Accuracy: 0.7163599729537964
Epoch 3, Loss: 0.010276271030306816, Accuracy: 0.9965999722480774, Test Loss: 1.725981593132019, Test_Accuracy: 0.7457600235939026
Epoch 4, Loss: 0.008528096601366997, Accuracy: 0.9972400069236755, Test Loss: 1.7245286703109741, Test_Accuracy: 0.7318400144577026
Epoch 5, Loss: 0.009184456430375576, Accuracy: 0.9972000122070312, Test Loss: 1.7929840087890625, Test_Accuracy: 0.7309600114822388
Epoch 6, Loss: 0.010907646268606186, Accuracy: 0.9964399933815002, Test Loss: 1.7789119482040405, Test_Accuracy: 0.7431600093841553
Epoch 7, Loss: 0.007512977812439203, Accuracy: 0.9974799752235413, Test Loss: 1.876678705215454, Test_Accuracy: 0.7249600291252136
Epoch 8, Loss: 0.007751119788736105, Accuracy: 0.9973599910736084, Test Loss: 1.89

# GRU (Gated Recurrent Units)
---

In [10]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        # just replacing SimpleRNN with GRU
        self.rnn = tf.keras.layers.GRU(32)
        self.dense = tf.keras.layers.Dense(2, activation = 'softmax')       
        
    def call(self, x, training = None, mask = None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

In [11]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer,
                  train_loss, train_accuracy)
        
    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object,
                 test_loss, test_accuracy)
        
    print(f'Epoch {epoch + 1}, ' +
         f'Loss: {train_loss.result()}, ' +
         f'Accuracy: {train_accuracy.result()}, ' +
         f'Test Loss: {test_loss.result()}, ' +
         f'Test_Accuracy: {test_accuracy.result()}')
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.0028990558348596096, Accuracy: 0.9991999864578247, Test Loss: 1.9898751974105835, Test_Accuracy: 0.7301999926567078
Epoch 2, Loss: 0.010745164006948471, Accuracy: 0.995959997177124, Test Loss: 1.9018616676330566, Test_Accuracy: 0.7278000116348267
Epoch 3, Loss: 0.013139204122126102, Accuracy: 0.9958000183105469, Test Loss: 1.7916804552078247, Test_Accuracy: 0.7285199761390686
Epoch 4, Loss: 0.003536507487297058, Accuracy: 0.9991599917411804, Test Loss: 1.980135202407837, Test_Accuracy: 0.7309600114822388
Epoch 5, Loss: 0.0017628748901188374, Accuracy: 0.9994800090789795, Test Loss: 2.0729358196258545, Test_Accuracy: 0.7313200235366821
Epoch 6, Loss: 0.002960232552140951, Accuracy: 0.9991599917411804, Test Loss: 2.0972747802734375, Test_Accuracy: 0.7120400071144104
Epoch 7, Loss: 0.016600821167230606, Accuracy: 0.9940400123596191, Test Loss: 2.004356622695923, Test_Accuracy: 0.7276800274848938
Epoch 8, Loss: 0.006795607507228851, Accuracy: 0.9982399940490723, Test Loss: